In [3]:
import tensorflow as tf
import numpy as np
import h2o
import os
import pandas as pd

# from .feature_engineering_filter import Find_correct_port

2024-10-15 19:13:13.370526: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
h2o.init(max_mem_size="4g")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.1" 2021-10-19; OpenJDK Runtime Environment Temurin-17.0.1+12 (build 17.0.1+12); OpenJDK 64-Bit Server VM Temurin-17.0.1+12 (build 17.0.1+12, mixed mode, sharing)
  Starting server from /Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/tmpn2a6pmko
  JVM stdout: /var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/tmpn2a6pmko/h2o_kristofferseyffarth_started_from_python.out
  JVM stderr: /var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/tmpn2a6pmko/h2o_kristofferseyffarth_started_from_python.err
  Server is running at http://127.0.0.1:54337
Connecting to H2O server at http://127.0.0.1:54337 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 16 days
H2O_cluster_name:,H2O_from_python_kristofferseyffarth_78w4r7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [5]:
train_data = h2o.import_file(
    path="../Datasets/ais_train.csv", sep="|", header=1
)  # time|cog|sog|rot|heading|navstat|etaRaw|latitude|longitude|vesselId|portId

test_data = h2o.import_file(
    path="../Datasets/ais_test.csv", sep=",", header=1
)  # ID,vesselId,time,scaling_factor

schedules = h2o.import_file(
    path="../Datasets/schedules_to_may_2024.csv", sep="|", header=1
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
from .preprocessing import Preprocessing

# Implement preprocessing

In [ ]:
train_data_df = train_data.as_data_frame()
schedules_df = schedules.as_data_frame()

In [7]:
# feature engineering
from h2o.frame import H2OFrame


def append_current_schedule(
    queries_frame: H2OFrame, schedules_frame: H2OFrame
) -> H2OFrame:
    schedules_frame_stripped = schedules_frame[
        ["vesselId", "portLatitude", "portLongitude", "arrivalDate", "sailingDate"]
    ]

    queries_frame_merged = queries_frame.merge(schedules_frame_stripped)
    print(queries_frame_merged.columns)

    queries_frame_merged["time"] = queries_frame_merged["time"].as_date("%Y-%m-%d %H:%M:%S")
    queries_frame_merged["arrivalDate"] = queries_frame_merged["arrivalDate"].as_date("%Y-%m-%d %H:%M:%S")
    queries_frame_merged["sailingDate"] = queries_frame_merged["sailingDate"].as_date("%Y-%m-%d %H:%M:%S")

    queries_frame_filtered = queries_frame_merged[
        (queries_frame_merged["time"] <= queries_frame_merged["arrivalDate"])
        & (queries_frame_merged["time"] >= queries_frame_merged["sailingDate"])
    ]
    return queries_frame_filtered


# implement feature engineering
train_data_appended = append_current_schedule(train_data, schedules)
print(train_data_appended.head())

KeyboardInterrupt: 

In [ ]:
train_data_appended.export_file(path="intermediate/train_data_with_schedule.csv", force=True)

In [ ]:
# Convert H2O Frame to pandas DataFrame
train_data_df = train_data.as_data_frame()

# Convert 'timestamp' column to datetime
train_data_df["time"] = pd.to_datetime(train_data_df["time"]).astype(int) / 10**9
train_data_df.drop(columns=["etaRaw"], inplace=True)


# Convert pandas DataFrame back to H2O Frame
train_data = h2o.H2OFrame(train_data_df)
print(train_data.head())

In [ ]:
# Convert H2O Frame to pandas DataFrame
test_data_df = test_data.as_data_frame()

# Convert 'timestamp' column to datetime
test_data_df["time"] = pd.to_datetime(test_data_df["time"]).astype(int) / 10**9

# Convert pandas DataFrame back to H2O Frame
test_data = h2o.H2OFrame(test_data_df)
print(train_data.head())

In [ ]:
splits = train_data.split_frame(
    ratios=[0.7], seed=1
)  # 70% for train_data, 30% for validation_data

# create a test subset from train data
train_data = splits[0]
validation_data = splits[1]

In [ ]:
print(test_data.columns)
test_data = test_data.drop("ID", axis=1)
test_data = test_data.drop("scaling_factor", axis=1)

In [ ]:
print(test_data.columns)

In [ ]:
features_test = [
    "time",
    "vesselId",
]

features_train = [
    "time",
    "vesselId",
    "cog",
    "sog",
    "rot",
    "heading",
    "navstat",
    # "etaRaw", #Remove etaRaw because it requires preprocessing
    # "portId",
]
target_long = "longitude"  # replace with your actual target column
target_lat = "latitude"  # replace with your actual target column

target_cog = "cog"
target_sog = "sog"
target_rot = "rot"
target_heading = "heading"
target_navstat = "navstat"
# target_etaRaw = "etaRaw" #Remove etaRaw because it requires preprocessing
target_portId = "portId"

In [ ]:
hyper_params = {"ntrees": [50, 100, 200], "learn_rate": [0.01, 0.1, 0.2, 0.3]}

In [ ]:
gbm_lat = h2o.estimators.H2OXGBoostEstimator()
gbm_long = h2o.estimators.H2OXGBoostEstimator()

gbm_cog = h2o.estimators.H2OXGBoostEstimator()
gbm_sog = h2o.estimators.H2OXGBoostEstimator()
gbm_rot = h2o.estimators.H2OXGBoostEstimator()
gbm_heading = h2o.estimators.H2OXGBoostEstimator()
gbm_navstat = h2o.estimators.H2OXGBoostEstimator()
# gbm_etaRaw = h2o.esti#mators.H2OXGBoostEstimator() #Remove etaRaw because it requires preprocessing
# gbm_portId = h2o.estimators.H2OXGBoostEstimator()

In [ ]:
gbm_cog.train(x=features_test, y=target_cog, training_frame=train_data)
cog_mse = gbm_cog.model_performance(validation_data).mse()
print("cog_mse: ", cog_mse)
cog_predicted = gbm_cog.predict(test_data)


h2o.remove(gbm_cog)

In [ ]:
gbm_sog.train(x=features_test, y=target_sog, training_frame=train_data)
sog_mse = gbm_sog.model_performance(validation_data).mse()
print("sog_mse: ", sog_mse)
sog_predicted = gbm_sog.predict(test_data)
h2o.remove(gbm_sog)

In [ ]:
gbm_rot.train(x=features_test, y=target_rot, training_frame=train_data)
rot_mse = gbm_rot.model_performance(validation_data).mse()
print("rot_mse: ", rot_mse)
rot_predicted = gbm_rot.predict(test_data)
h2o.remove(gbm_rot)

In [ ]:
gbm_heading.train(x=features_test, y=target_heading, training_frame=train_data)
heading_mse = gbm_heading.model_performance(validation_data).mse()
print("heading_mse: ", heading_mse)
heading_predicted = gbm_heading.predict(test_data)
h2o.remove(gbm_heading)

In [ ]:
train_data[target_navstat] = train_data[target_navstat].asfactor()

gbm_navstat.train(x=features_test, y=target_navstat, training_frame=train_data)
navstat_performance = gbm_navstat.model_performance(validation_data)
navstat_predicted = gbm_navstat.predict(test_data)

In [ ]:
validation_data[target_navstat] = validation_data[target_navstat].asfactor()
navstat_performance = gbm_navstat.model_performance(validation_data)

print("navstat_precision: ", navstat_performance.mse())
print("navstat_recall: ", navstat_performance.recall())
print("navstat_f1: ", navstat_performance.F1())
print("navstat_auc: ", navstat_performance.auc())

In [ ]:
# gbm_etaRaw.train(x=features_test, y=target_etaRaw, training_frame=train_data)
# etaRaw_predicted = gbm_etaRaw.predict(test_data)
# h2o.remove(gbm_etaRaw)

In [ ]:
# gbm_portId.train(x=features_test, y=target_portId, training_frame=train_data)
# portId_predicted = gbm_portId.predict(test_data)
# h2o.remove(gbm_portId)

In [ ]:
gbm_lat.train(x=features_train, y=target_lat, training_frame=train_data)
gbm_long.train(x=features_train, y=target_long, training_frame=train_data)

In [ ]:
cog_predicted.set_names(["cog"])
sog_predicted.set_names(["sog"])
rot_predicted.set_names(["rot"])
heading_predicted.set_names(["heading"])
navstat_predicted.set_names(["navstat"])
# etaRaw_predicted.set_names(["etaRaw"])
# portId_predicted.set_names(["portId"])

In [ ]:
test_data_predicted = test_data
test_data_predicted = test_data.cbind(cog_predicted)
test_data_predicted = test_data.cbind(sog_predicted)
test_data_predicted = test_data.cbind(rot_predicted)
test_data_predicted = test_data.cbind(heading_predicted)
test_data_predicted = test_data.cbind(navstat_predicted)
# test_data_predicted = test_data.cbind(etaRaw_predicted)
# test_data_predicted = test_data.cbind(portId_predicted)

In [ ]:
# grid_lat = h2o.grid.grid_search.H2OGridSearch(gbm_lat, hyper_params)
# grid_lat.train(x=features, y=target_lat, training_frame=train_data)

In [ ]:
# gridperf = grid_lat.get_grid(sort_by="mse", decreasing=True)

# gbm_lat = gridperf.models[0]

In [ ]:
# print(gbm_lat.params["learn_rate"]["actual"])
# print(gbm_lat.params["ntrees"]["actual"])

# 0.01
# 50

In [ ]:
# gbm_long = h2o.estimators.H2OXGBoostEstimator(
#     learn_rate=gbm_lat.params["learn_rate"]["actual"],
#     ntrees=gbm_lat.params["ntrees"]["actual"],
# )

In [ ]:
# gbm_long.train(x=features, y=target_long, training_frame=train_data)

In [ ]:
lat_predictions = gbm_lat.predict(test_data_predicted)
long_predictions = gbm_long.predict(test_data_predicted)

In [ ]:
lat_predictions_df = lat_predictions.as_data_frame()
long_predictions_df = long_predictions.as_data_frame()

In [ ]:
predictions_df = pd.concat([lat_predictions_df, long_predictions_df], axis=1)
predictions_df.columns = ["latitude_predicted", "longitude_predicted"]

In [ ]:
predictions_df["ID"] = predictions_df.index
# set the ID as the first column
predictions_df = predictions_df[["ID", "longitude_predicted", "latitude_predicted"]]

In [ ]:
predictions_df.to_csv("predictions.csv", index=False)